<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>python-awips: Working with Models</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

<div style="float:right; width:250 px"><img src="../../instructors/images/python-awips-model-preview.png" alt="Colorized RAP13 CONUS image" style="height: 300px;"></div>


### Focuses

* Create a colorized plot for the continental US of model data (grib).
* Access the model data from an EDEX server and limit the data returned by using model specific parameters.
* Use both *pcolormesh* and *contourf* to create colorized plots, and compare the differences between the two.


### Objectives

1. [Define Data Request](#1.-Define-Data-Request)
1. [Limit Results Based on Time](#2.-Limit-Results-Based-on-Time)
1. [Define Map Properties](#3.-Define-Map-Properties)
1. [Use the Grid Data!](#4.-Use-the-Grid-Data!)
1. [Plot Using pcolormesh](#5.-Plot-Using-pcolormesh)
1. [Plot Using contourf](#6.-Plot-Using-contourf)

---

### Imports

In [ ]:
from awips.dataaccess import DataAccessLayer
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy.constants import convert_temperature

## 1. Define Data Request

If you read through the [python-awips: How to Access Data](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/dataAccess/python-awips-HowToAccessData.ipynb) training, you will know that we need to set an EDEX url to access our server, and then we create a data request.  In this example we use *grid* as the data type to define our request.
In addition to setting the data type, the location, parameters and levels are also set as **RAP13**, **T** (for temperature), and **2.0FHAG** (for Fixed Height Above Ground in meters), respectively.

In [ ]:
DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest()
request.setDatatype("grid")
request.setLocationNames("RAP13")
request.setParameters("T")
request.setLevels("2.0FHAG")

# Take a look at our request
print(request)

<a href="#top">Top</a>

---

## 2. Limit Results Based on Time

Models produce many different time variants during their runs, so let's limit the data to the most recent time and forecast run.

> Note: You can play around with different times and forecast runs to see the differences.

In [ ]:
cycles = DataAccessLayer.getAvailableTimes(request, True)
times = DataAccessLayer.getAvailableTimes(request)
fcstRun = DataAccessLayer.getForecastRun(cycles[-1], times)

# Get the most recent grid data
response = DataAccessLayer.getGridData(request, [fcstRun[0]])

print('Number of available times:', len(times))
print('Number of available forecast runs:', len(fcstRun))

<a href="#top">Top</a>

---

## 3. Define Map Properties

In order to plot more than one image, it's easiest to define common logic in a function.  Here, a new function called **make_map** is defined.  This function uses the [matplotlib.pyplot package (plt)](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.html) to create a figure and axis.  The coastlines (continental boundaries) are added, along with lat/lon grids.  

In [ ]:
def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(16, 9),
            subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.coastlines(resolution='50m')
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

<a href="#top">Top</a>

---

## 4. Use the Grid Data!

Here we get our grid data object from our previous response, and then get the raw data array off that object.  We also get the latitude and longitude arrays, and create a bounding box that we'll use when creating our plots (by calling **make_map** defined above).  Finally, we'll convert our data from degrees Kelvin to Farenheit to make the plot more understandable.

In [ ]:
grid = response[0]
data = grid.getRawData()
lons, lats = grid.getLatLonCoords()
bbox = [lons.min(), lons.max(), lats.min(), lats.max()]

# Convert temp from Kelvin to F
destUnit = 'F'
data = convert_temperature(data, 'K', destUnit)

<a href="#top">Top</a>

---

## 5. Plot Using pcolormesh

This example shows how to use [matplotlib.pyplot.pcolormesh](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.pcolormesh.html) to create a colorized plot. We use our **make_map** function to create a subplot and then we create a color scale (cs) and colorbar (cbar) with a label for our plot.

> Note: You may see a warning appear with a red background, this is okay, and will go away with subsequent runs of the cell.

In [ ]:
cmap = plt.get_cmap('rainbow')
fig, ax = make_map(bbox=bbox)
cs = ax.pcolormesh(lons, lats, data, cmap=cmap)
cbar = fig.colorbar(cs, shrink=0.7, orientation='horizontal')
cbar.set_label(grid.getLocationName() +" "+ grid.getLevel() + " " \
               + grid.getParameter() + " (" + destUnit + ") " \
               + "valid " + str(grid.getDataTime().getRefTime()))

<a href="#top">Top</a>

---

## 6. Plot Using contourf

This example shows how to use [matplotlib.pyplot.contourf](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.contourf.html) to create a colorized plot. We use our **make_map** function to create a subplot and then we create a color scale (cs2) and colorbar (cbar2) with a label for our plot.

In [ ]:
fig2, ax2 = make_map(bbox=bbox)
cs2 = ax2.contourf(lons, lats, data, 80, cmap=cmap,
                  vmin=data.min(), vmax=data.max())
cbar2 = fig2.colorbar(cs2, shrink=0.7, orientation='horizontal')
cbar2.set_label(grid.getLocationName() +" "+ grid.getLevel() + " " \
               + grid.getParameter() + " (" + destUnit + ") " \
               + "valid " + str(grid.getDataTime().getRefTime()))

<a href="#top">Top</a>

---

## See also

Documentation for:

* [awips.DataAccessLayer](http://unidata.github.io/python-awips/api/DataAccessLayer.html)
* [awips.PyGridData](http://unidata.github.io/python-awips/api/PyGridData.html)
* [matplotlib.pyplot](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.html)
* [matplotlib.pyplot.subplot](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.subplot.html)
* [matplotlib.pyplot.pcolormesh](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.pcolormesh.html)
* [matplotlib.pyplot.contourf](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.contourf.html)


### Related Notebooks

* [python-awips: How to Access Data](https://nbviewer.jupyter.org/github/Unidata/pyaos-ams-2021/blob/master/notebooks/dataAccess/python-awips-HowToAccessData.ipynb)


<a href="#top">Top</a>